## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,2021-03-28 00:58:47,-33.6500,115.3333,66.99,71,0,1.01,light rain
1,1,El Tocuyo,VE,2021-03-28 00:59:32,9.7873,-69.7937,71.98,78,100,2.80,overcast clouds
2,2,Rikitea,PF,2021-03-28 00:59:32,-23.1203,-134.9692,79.48,72,0,17.65,clear sky
3,3,Sitka,US,2021-03-28 00:59:32,57.0531,-135.3300,37.40,75,75,16.11,snow
4,4,Dalmeny,CA,2021-03-28 00:59:33,52.3334,-106.7679,41.00,33,75,9.22,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,2021-03-28 00:59:32,-23.1203,-134.9692,79.48,72,0,17.65,clear sky
6,6,Ballina,AU,2021-03-28 00:59:33,-28.8667,153.5667,78.80,57,0,8.05,clear sky
8,8,Saint-Philippe,RE,2021-03-28 00:59:34,-21.3585,55.7679,75.20,83,20,5.75,few clouds
11,11,Porbandar,IN,2021-03-28 00:59:34,21.6422,69.6093,77.22,65,59,10.00,broken clouds
21,21,Victoria,HK,2021-03-28 00:59:03,22.2855,114.1577,75.00,88,8,4.25,clear sky
22,22,Phan Thiet,VN,2021-03-28 00:59:36,10.9333,108.1000,81.43,65,30,8.05,scattered clouds
30,30,Lorengau,PG,2021-03-28 00:59:38,-2.0226,147.2712,83.86,73,91,9.37,overcast clouds
31,31,Bambous Virieux,MU,2021-03-28 00:59:38,-20.3428,57.7575,77.00,94,75,5.75,broken clouds
32,32,Cayenne,GF,2021-03-28 00:59:38,4.9333,-52.3333,80.01,94,75,11.81,broken clouds
35,35,Vaini,TO,2021-03-28 00:59:11,-21.2000,-175.2000,86.00,74,75,8.05,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Date                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.count()

City_ID                210
City                   210
Country                210
Date                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.48,clear sky,-23.1203,-134.9692,
6,Ballina,AU,78.80,clear sky,-28.8667,153.5667,
8,Saint-Philippe,RE,75.20,few clouds,-21.3585,55.7679,
11,Porbandar,IN,77.22,broken clouds,21.6422,69.6093,
21,Victoria,HK,75.00,clear sky,22.2855,114.1577,
22,Phan Thiet,VN,81.43,scattered clouds,10.9333,108.1000,
30,Lorengau,PG,83.86,overcast clouds,-2.0226,147.2712,
31,Bambous Virieux,MU,77.00,broken clouds,-20.3428,57.7575,
32,Cayenne,GF,80.01,broken clouds,4.9333,-52.3333,
35,Vaini,TO,86.00,light rain,-21.2000,-175.2000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.48,clear sky,-23.1203,-134.9692,Pension Maro'i
6,Ballina,AU,78.80,clear sky,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
8,Saint-Philippe,RE,75.20,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Porbandar,IN,77.22,broken clouds,21.6422,69.6093,Lords Inn Porbandar
21,Victoria,HK,75.00,clear sky,22.2855,114.1577,Mini Hotel Central
...,...,...,...,...,...,...,...
666,Sulangan,PH,81.73,scattered clouds,11.1373,123.7228,Maia's Beach Resort
680,Ende,ID,84.42,overcast clouds,-8.8432,121.6623,Dwi Putra Hotel - Ende
687,Belmonte,BR,78.80,clear sky,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
688,Kaliganj,BD,78.19,clear sky,23.4095,89.1402,Sohag sohan hostel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = clean_hotel_df["Max Temp"]
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))